Вам даны данные следующего формата: для каждого события зафиксирован идентификатор пользователя (user_id) и сумма транзакции (gmv). Поскольку один пользователь может совершить несколько транзакций, исходные данные представляют собой детализированные (пособытийные), а не агрегированные (поюзерные) данные.

Сагрегируйте данные до пользователя (тип агрегации: сумма). Найдите несмещенную оценку матожидания (выборочное среднее) и дисперсии.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("synthetic_gmv_data_1.1.csv")
df.head()

,user_id,gmv
0,guo0pdwqvg,506
1,guo0pdwqvg,760
2,guo0pdwqvg,633
3,guo0pdwqvg,506
4,guo0pdwqvg,760


In [3]:
df_user = df.groupby("user_id").agg({"gmv": "sum"}).reset_index()
df_user.head()

,user_id,gmv
0,0001ffmxo2,3766
1,000c2glbkl,2456
2,000c5yxi7t,700
3,000eg9ozf2,5082
4,000h7yt1ve,2539


In [4]:
round(df_user.gmv.mean(), 3), round(df_user.gmv.var(ddof=1), 3)

(np.float64(2750.847), np.float64(3646228.409))

---

Используйте оценку матожидания и оценку дисперсии из предыдущей задачи для дизайна эксперимента.

Предположим, мы хотим "увидеть" эффект в 1% с вероятностью ошибки первого рода 5% и вероятностью ошибки второго рода 20%.

Разбиение на тестовую и контрольную группу у нас 1 к 3 (контроль в 3 раза больше теста). Предположим, что дисперсии в тестовой и контрольной группах одинаковые.

Сколько всего пользователей должно быть в эксперименте?

In [5]:
import math

from scipy import stats

In [6]:
def size_mde(sigma, effect, k = 1, alpha = 0.05, beta = 0.2):
    dist = stats.norm(loc=0, scale=1)
    const = (dist.ppf(1- alpha/2) + dist.ppf(1- beta))**2
    var = sigma**2 
    return const*(1+k)*var/(effect**2)

In [7]:
s = size_mde(sigma=df_user.gmv.std(ddof=1), effect=df_user.gmv.mean()*0.01, k = 3, alpha=0.05, beta=0.2)
math.ceil(s + s / 3)

201706

---

Вам даны данные следующего формата: для каждого события зафиксирован идентификатор пользователя (user_id), сумма транзакции (gmv), группа эксперимента (group_name).

Сагрегируйте данные до пользователя (тип агрегации: сумма). Мы проверяем гипотезу равенства средних поюзерных gmv.

Посчитайте T-статистику. Найдите P-value, используя T-test.

Используйте данные из файла synthetic_gmv_data_1.2.csv

NB! Здесь и далее, если вы будете использовать встроенную функцию stats.ttest_ind, не забудьте добавить параметр equal_var=False. Эта функция использует поправки Уэлча на степени свободы, когда мы считаем "руками", то обычно используем упрощенную формулу для степеней свободы m+n−2m+n−2 (количество уников теста + количество уников контроля - 2). Поэтому ответы могут немного различаться. Оба варианта будут считаться верными.

In [154]:
df = pd.read_csv("synthetic_gmv_data_1.2.csv")
df.head()

,user_id,gmv,group_name
0,myo4ixol31,1428,test
1,myo4ixol31,1428,test
2,myo4ixol31,1071,test
3,myo4ixol31,1071,test
4,pkzf2889ww,351,test


In [9]:
df_user = df.groupby(["user_id", "group_name"], as_index=False).agg({"gmv": "sum"})
df_user.head()

,user_id,group_name,gmv
0,00062h7u56,control,733
1,00074uxybk,test,3187
2,000ic5j18m,control,2933
3,000plmykri,test,1695
4,00174ganru,control,1496


In [10]:
df_user.loc[df_user.group_name=='test'].gmv.mean(), df_user.loc[df_user.group_name=='control'].gmv.mean()

(np.float64(2870.650366598778), np.float64(2847.2169990019756))

In [11]:
t_statistic, p_value = stats.ttest_ind(df_user.loc[df_user.group_name=='test'].gmv, df_user.loc[df_user.group_name=='control'].gmv, equal_var=False)
round(t_statistic, 3), round(p_value, 3)

(np.float64(2.36), np.float64(0.018))

---

На тех же данных теперь проверим гипотезу равенства среднего чека.

Посчитайте T-статистику, найдите P-value.

In [12]:
import numpy as np

In [13]:
def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return np.nan

In [14]:
df_user = df.groupby(["user_id", "group_name"], as_index=False).agg({"gmv": ["sum", "count"]})
df_user.head()

user_id group_name   gmv      
                           sum count
0  00062h7u56    control   733     1
1  00074uxybk       test  3187     3
2  000ic5j18m    control  2933     6
3  000plmykri       test  1695     5
4  00174ganru    control  1496     4

In [238]:
def delta_var(numerator, denominator):
    """
    Функция для расчета дисперсии дельта-методом, numerator - вектор числитель, denominator - вектор знаменатель
    """
    x = numerator
    y = denominator
    n = len(x)
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0][1]    
    delta_var = safe_divide(safe_divide(var_x,mu_y**2)  - cov_xy*safe_divide(mu_x,mu_y**3) + var_y*safe_divide(mu_x**2,mu_y**4), n)
    return delta_var

In [16]:
df_user.loc[df_user.group_name=='test', [('gmv', 'sum')]]

,gmv
,sum
1,3187
3,1695
8,1293
9,3862
11,765
...,...
196372,3809
196377,3938
196384,3422


In [17]:
n = len(df_user.loc[df_user.group_name=='test'])
m = len(df_user.loc[df_user.group_name=='control'])
n, m

(49100, 147291)

In [18]:
test_var = delta_var(df_user.loc[df_user.group_name=='test', [('gmv', 'sum')]].values.flatten(), df_user.loc[df_user.group_name=='test', [('gmv', 'count')]].values.flatten())
test_var

np.float64(2.1727173707917586)

In [19]:
control_var = delta_var(df_user.loc[df_user.group_name=='control', [('gmv', 'sum')]].values.flatten(), df_user.loc[df_user.group_name=='control', [('gmv', 'count')]].values.flatten())
control_var

np.float64(0.714227879688474)

In [20]:
np.mean(df_user.loc[df_user.group_name=='test', [('gmv', 'sum')]])

np.float64(2870.650366598778)

In [21]:
sigma = np.sqrt(test_var + control_var)
delta_estimator = safe_divide(np.mean(df_user.loc[df_user.group_name=='test', [('gmv', 'sum')]]), np.mean(df_user.loc[df_user.group_name=='test', [('gmv', 'count')]])) - safe_divide(np.mean(df_user.loc[df_user.group_name=='control', [('gmv', 'sum')]]), np.mean(df_user.loc[df_user.group_name=='control', [('gmv', 'count')]]))
tt = safe_divide(delta_estimator, sigma)
pvalue = 2*stats.t.sf(np.abs(tt),n+m-2)

In [22]:
round(tt, 3), round(pvalue, 3)

(np.float64(2.344), np.float64(0.019))

---

На тех же данных теперь проверим гипотезу равенства среднего чека с использованием линеаризации второго типа и классический t-test:

Перейдем к линеаризованной метрике $L(u)=X/Y+1/Y*X(u)−¯¯¯X¯¯¯Y2Y(u)$, где

$X(u)$ - значение числителя для пользователя uu

$Y(u)$ - значение знаменателя для пользователя uu

¯¯¯XXˉ - среднее в группе по числителю

¯¯¯YYˉ - среднее в группе по знаменателю

Посчитайте T-статистику, найдите P-value для линеаризованной метрики.

In [73]:
def linearization_type_2(x_num, x_denom, y_num, y_denom):
    n = len(x_num)
    m = len(y_num)
    x_num_bar = np.mean(x_num)
    y_num_bar = np.mean(y_num)
    x_denom_bar = np.mean(x_denom)
    y_denom_bar = np.mean(y_denom)
    x_estimator = safe_divide(x_num_bar, x_denom_bar)
    y_estimator = safe_divide(y_num_bar, y_denom_bar)
    delta_estimator = x_estimator - y_estimator
    x_linear = x_estimator  + safe_divide(1, x_denom_bar)*(np.array(x_num) - x_estimator*np.array(x_denom))
    y_linear = y_estimator  + safe_divide(1, y_denom_bar)*(np.array(y_num) - y_estimator*np.array(y_denom))
    t_stat, p_value = stats.ttest_ind(x_linear, y_linear, equal_var=False)
    is_sig = 1
    if p_value > 0.05:
        is_sig = 0
    return t_stat, p_value

In [74]:
x_num = df_user.loc[df_user.group_name=='test', [('gmv', 'sum')]].values.flatten()
x_denom = df_user.loc[df_user.group_name=='test', [('gmv', 'count')]].values.flatten()
y_num = df_user.loc[df_user.group_name=='control', [('gmv', 'sum')]].values.flatten()
y_denom = df_user.loc[df_user.group_name=='control', [('gmv', 'count')]].values.flatten()

In [76]:
tt, pvalue_linearization_type_2 = linearization_type_2(x_num, x_denom, y_num, y_denom)
round(tt, 3), round(pvalue_linearization_type_2, 3)

(np.float64(2.344), np.float64(0.019))

---

На тех же данных постройте доверительные интервалы для:

Δ gmv - разницы средних gmv

Δ gmv, % - процентного изменения средних gmv

Δ aov - разницы средних чеков

Δ aov, % - процентного изменения средних чеков

<img src="Screenshot 2025-07-27 at 15.57.52.png" width="800">


In [239]:
df_user = df.groupby(["user_id", "group_name"], as_index=False).agg({"gmv": "sum"})
df_user.head()

,user_id,group_name,gmv
0,00062h7u56,control,733
1,00074uxybk,test,3187
2,000ic5j18m,control,2933
3,000plmykri,test,1695
4,00174ganru,control,1496


In [240]:
alpha = 0.05
n = df_user.loc[df_user.group_name=='test'].gmv.shape[0]
m = df_user.loc[df_user.group_name=='control'].gmv.shape[0]

delta = df_user.loc[df_user.group_name=='test'].gmv.mean() - df_user.loc[df_user.group_name=='control'].gmv.mean()
sigma = np.sqrt(df_user.loc[df_user.group_name=='test'].gmv.var(ddof=1) / n + df_user.loc[df_user.group_name=='control'].gmv.var(ddof=1) / m)

[round((delta - stats.t.ppf(1-alpha/2, df=n+m-2) * sigma), 5), round(delta + stats.t.ppf(1-alpha/2, df=n+m-2) * sigma, 5)]

[np.float64(3.97539), np.float64(42.89134)]

In [241]:
stats.t.interval(0.95, df=n+m-2, loc=delta, scale=sigma)

(np.float64(3.9753946147033155), np.float64(42.891340578901335))

In [242]:
n = df_user.loc[df_user.group_name=='test'].gmv.shape[0]
m = df_user.loc[df_user.group_name=='control'].gmv.shape[0]

x_num = df_user.loc[df_user.group_name=='test'].gmv
y_num = df_user.loc[df_user.group_name=='control'].gmv

var_x = np.var(x_num, ddof=1)
var_y = np.var(y_num, ddof=1)  
mu_x = np.mean(x_num)
mu_y = np.mean(y_num)
delta_var_ = safe_divide(var_x, n * mu_y**2) + safe_divide(var_y*mu_x**2, m * mu_y**4)
sigma = np.sqrt(delta_var_)
uplift_estimator = ((mu_x - mu_y)/mu_y) * 100
[round(uplift_estimator - 100*stats.t.ppf(1-alpha/2, df=n+m-2) * sigma, 5), round(uplift_estimator + 100*stats.t.ppf(1-alpha/2, df=n+m-2) * sigma, 5)]

[np.float64(0.13824), np.float64(1.50782)]

In [230]:
stats.t.interval(0.95, df=n+m-2, loc=uplift_estimator, scale=sigma*100)

(np.float64(0.13823808422869932), np.float64(1.5078161227401643))

In [243]:
df_user = df.groupby(["user_id", "group_name"], as_index=False).agg({"gmv": ["sum", "count"]})
df_user.head()

user_id group_name   gmv      
                           sum count
0  00062h7u56    control   733     1
1  00074uxybk       test  3187     3
2  000ic5j18m    control  2933     6
3  000plmykri       test  1695     5
4  00174ganru    control  1496     4

In [244]:
x_num = df_user.loc[df_user.group_name=='test', [('gmv', 'sum')]].values.flatten()
x_denom = df_user.loc[df_user.group_name=='test', [('gmv', 'count')]].values.flatten()
y_num = df_user.loc[df_user.group_name=='control', [('gmv', 'sum')]].values.flatten()
y_denom = df_user.loc[df_user.group_name=='control', [('gmv', 'count')]].values.flatten()

In [245]:
n = len(x_num)
m = len(y_num)
test_var = delta_var(x_num, x_denom)
control_var = delta_var(y_num, y_denom)
sigma = np.sqrt(test_var + control_var)
delta_estimator = safe_divide(np.mean(x_num), np.mean(x_denom)) - safe_divide(np.mean(y_num), np.mean(y_denom))
[delta_estimator - stats.t.ppf(1-alpha/2, df=n+m-2) * sigma, delta_estimator + stats.t.ppf(1-alpha/2, df=n+m-2) * sigma]

[np.float64(-0.7972906933246167), np.float64(8.762242156670485)]

In [246]:
stats.t.interval(0.95, df=n+m-2, loc=delta_estimator, scale=sigma)

(np.float64(-0.7972906933246167), np.float64(8.762242156670485))

In [247]:
n = len(x_num)
m = len(y_num)
test_var = delta_var(x_num, x_denom)
control_var = delta_var(y_num, y_denom)
R_t = safe_divide(np.mean(x_num), np.mean(x_denom))
R_c = safe_divide(np.mean(y_num), np.mean(y_denom))
delta_estimator = ((R_t - R_c) / R_c) * 100
sigma = np.sqrt(safe_divide(test_var,(R_c**2)) + safe_divide((control_var*(R_t**2)),(R_c**4)))
[round(delta_estimator - 100*stats.t.ppf(1-alpha/2, df=n+m-2) * sigma, 5), round(delta_estimator + 100*stats.t.ppf(1-alpha/2, df=n+m-2) * sigma, 6)]

[np.float64(-0.11482), np.float64(1.252309)]

In [248]:
stats.t.interval(0.95, df=n+m-2, loc=delta_estimator, scale=sigma*100)


(np.float64(-0.11482161875509334), np.float64(1.2523086903892158))

---

Вам даны данные следующего формата: поюзерный исторический gmv (gmv_hist), поюзерный gmv во время эксперимента (gmv_exp), группа эксперимента (group_name).

Посчитайте коэффициент θ=Cov(X,Y)VarXθ=VarXCov(X,Y)​, используя все данные и теста, и контроля), где XX - gmv_hist, YY - gmv_exp.

Найдите сuped значение метрики.

Посчитайте P-value без применения CUPED и P-value c применением CUPED

In [129]:
df = pd.read_csv("synthetic_gmv_data_1.3.csv")
df.head()

,gmv_hist,gmv_exp,group_name
0,200.78,123.19,test
1,363.80,134.49,control
2,39.93,116.72,control
3,150.99,177.67,control
4,208.93,65.30,test


In [133]:
theta = np.cov(df.gmv_hist, df.gmv_exp)[0][1] / df.gmv_hist.var(ddof=1)
theta

np.float64(0.5578881785090701)

In [134]:
Z = df.gmv_exp - theta * df.gmv_hist + theta * df.gmv_hist.mean()
Z

0         111.267662
1          31.620732
2         194.533976
3         193.524915
4          48.830874
             ...    
249995     41.451801
249996    111.754257
249997    252.109044
249998    179.266108
249999    141.568223
Length: 250000, dtype: float64

In [137]:
np.var(Z, ddof=1), np.var(df.gmv_exp, ddof=1) * (1 - np.corrcoef(df.gmv_exp, df.gmv_hist)[0][1]**2)

(np.float64(6935.0616580628575), np.float64(6935.061658062852))

In [145]:
df.loc[df.group_name=='test', ['gmv_hist']].shape

(124838, 1)

In [146]:
df.loc[df.group_name=='control', ['gmv_hist']].shape

(125162, 1)

In [143]:
X_pre = df.loc[df.group_name=='test', ['gmv_hist']] - df.loc[df.group_name=='control', ['gmv_hist']]
X_pre

,gmv_hist
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
249995,NaN
249996,NaN
249997,NaN
249998,NaN


In [139]:
Y = df.loc[df.group_name=='test', ['gmv_exp']].mean() - df.loc[df.group_name=='control', ['gmv_exp']].mean()
Y

gmv_exp    0.533918
dtype: float64

In [148]:
Y_cuped = df.loc[:, 'gmv_exp'] - theta * df.loc[:, 'gmv_hist']
Y_cuped

0          11.177212
1         -68.469719
2          94.443525
3          93.434464
4         -51.259577
             ...    
249995    -58.638650
249996     11.663806
249997    152.018593
249998     79.175657
249999     41.477772
Length: 250000, dtype: float64

In [149]:
group_control = (df.group_name == 'control')
group_test = (df.group_name == 'test')

In [151]:
_, pvalue_tt = stats.ttest_ind(df.loc[df.group_name=='test', 'gmv_exp'], df.loc[df.group_name=='control', 'gmv_exp'])
round(pvalue_tt, 3)

np.float64(0.233)

In [152]:
_, pvalue_cuped = stats.ttest_ind(Y_cuped[group_test], Y_cuped[group_control], equal_var=False)
round(pvalue_cuped, 3)

np.float64(0.047)